In [5]:
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

# Now you can import the TimeSeriesDataset class
from TimeSeriesDataset import TimeSeriesDataset as TSD

In [6]:
# initialize dataset by downloading dataset or downloading the data from polity_url
dataset = TSD(categories=['sc'])
# download all datasets
dataset.download_all_categories()

Downloaded 839 rows
Downloaded 464 rows
Downloaded sc/polity-territories dataset with 464 rows in 39.631311893463135 seconds
Downloaded 440 rows
Downloaded sc/polity-populations dataset with 440 rows in 46.26707911491394 seconds
Downloaded 492 rows
Downloaded sc/population-of-the-largest-settlements dataset with 492 rows in 40.329955101013184 seconds
Downloaded 474 rows
Downloaded sc/settlement-hierarchies dataset with 474 rows in 29.565491914749146 seconds
Downloaded 0 rows
Downloaded sc/administrative-levels dataset with 0 rows in 1.2806642055511475 seconds
Empty dataset for sc/administrative-levels
Downloaded 351 rows
Downloaded sc/religious-levels dataset with 351 rows in 28.29832434654236 seconds
Downloaded 364 rows
Downloaded sc/military-levels dataset with 364 rows in 17.50086522102356 seconds
Downloaded 381 rows
Downloaded sc/professional-military-officers dataset with 381 rows in 17.62206792831421 seconds
Downloaded 409 rows
Downloaded sc/professional-soldiers dataset with 409

In [8]:
# clean up dataser and create a debug dataset dataset.debug
dataset.debug_clean_dataset()

Polity gb_british_emp_2 (1900.0 - 1900.0) is inactive but has data for sc/polity-territories from 1800.0 to 1800.0
Polity gb_british_emp_2 (1900.0 - 1900.0) is inactive but has data for sc/polity-territories_from from 1800.0 to 1800.0
Polity gb_british_emp_2 (1900.0 - 1900.0) is inactive but has data for sc/polity-territories_to from 1800.0 to 1800.0
Polity jp_ashikaga (1400.0 - 1400.0) is inactive but has data for sc/polity-populations from 1500.0 to 1500.0
Polity us_early_illinois_confederation (1700.0 - 1700.0) is inactive but has data for sc/polity-populations from 1800.0 to 1800.0
Polity jp_ashikaga (1400.0 - 1400.0) is inactive but has data for sc/polity-populations_from from 1500.0 to 1500.0
Polity us_early_illinois_confederation (1700.0 - 1700.0) is inactive but has data for sc/polity-populations_from from 1800.0 to 1800.0
Polity jp_ashikaga (1400.0 - 1400.0) is inactive but has data for sc/polity-populations_to from 1500.0 to 1500.0
Polity us_early_illinois_confederation (1700

/Users/mperuzzo/Documents/repos/SeshatDatasetAnalysis/src/TimeSeriesDataset.py:275: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.raw.drop(columns = ['PolityActive'], inplace=True)


In [12]:
# if you try to debug the dataset twice it will inform you
dataset.debug_clean_dataset()
# save dataset
dataset.save_dataset(path = "../datasets")

Dataset already cleaned


In [13]:
# import dataset
dataset.load_dataset(path = "../datasets")


# remove all rows that have less than 30% of the columns filled in
dataset.remove_incomplete_rows(nan_threshold=0.3)
# build the social complexity variables
dataset.build_social_complexity()
# impute missing data
dataset.impute_missing_values()
print("Done")

KeyError: "['sc/administrative-levels', 'sc/canals'] not in index"